In [ ]:
install.packages("purrr")
install.packages("httr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(httr)
library(purrr)
library(dplyr)
library(tidyverse)
#library(RPostgreSQL)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ readr     2.1.5     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
obter_dfp <- function(ano){
  url <- paste0("https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/","dfp_cia_aberta_",ano,".zip") #copiando a URL dos arquivos
  GET(url, write_disk(paste0("dfp_cia_aberta_",ano,".zip"))) #baixando os arquivos
  planilhas <- unzip(paste0("/content/dfp_cia_aberta_",ano,".zip"))} #descompactando os arquivos .RAR

In [ ]:
obter_cias <- function(){
  #Listando os arquivos no diretório referente às demonstrações financeiras
  CIA <- list.files(path = ".", pattern = "^dfp_cia_aberta_20", full.names = TRUE)

  #Lendo os arquivos
  CIA <- read_csv2(CIA, id = "file_name",locale = locale(encoding = "ISO-8859-1"))

  #Obtendo o código e o nome das empresas
  CIA <- CIA %>% select(CD_CVM,DENOM_CIA) %>% distinct(CD_CVM,DENOM_CIA)

  #Convertendo para dataframe
  CIAS <- as.data.frame(CIA)
  names(CIAS) <- c("cd_cvm","nome_companhia")
  CIAS <- CIAS %>% distinct(cd_cvm, .keep_all = TRUE)

  #Salvando os arquivo gerados e deletando os outros
  write.csv(CIAS,"cias.csv")
  f <- list.files(path = ".", pattern = "^dfp_cia_aberta", full.names = TRUE, recursive = TRUE)
  file.remove(f)

  return(CIAS)
}

In [ ]:
anos <- 2010:2022

for (ano in anos){
obter_dfp(ano)}

In [ ]:
cias <- obter_cias()

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Multiple files in zip: reading 'dfp_cia_aberta_2010.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2011.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2012.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2013.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2014.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2015.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2016.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2017.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2018.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2019.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2020.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2021.csv'
Multiple files in zip: reading 'dfp_cia_aberta_2022.csv'
Rows: 22920 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (5): CNPJ_CIA, DENOM_CIA, CD_CVM, CATEG_DOC

In [ ]:
head(cias)

,CD_CVM,DENOM_CIA
,<chr>,<chr>
1,001023,BCO BRASIL S.A.
2,014206,BRB BCO DE BRASILIA S.A.
3,002437,CENTRAIS ELET BRAS S.A. - ELETROBRAS
4,014451,CIA ENERGETICA DE BRASILIA
5,014540,VBC ENERGIA SA
6,014621,NET SERVIÇOS DE COMUNICAÇÃO S.A.


In [ ]:
install.packages("writexl")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
#Conversão para excel
library(writexl)
write_xlsx(cias,"/content/cias.xlsx")

In [ ]:
# Estabelecer conexão com o banco de dados PostgreSQL
con <- dbConnect(PostgreSQL(), dbname = "seu_banco_de_dados",
                 host = "seu_host", port = "sua_porta",
                 user = "seu_usuario", password = "sua_senha")


# Inserir todas as linhas do dataframe no PostgreSQL
dbWriteTable(conn = con, name = "tabela_bd", value = cias,
             row.names = FALSE, overwrite = TRUE, append = FALSE)

# Fechar a conexão com o banco de dados
dbDisconnect(con)